<a href="https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/How_to_evaluate_Longformer_on_TriviaQA_using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation of a model using 🤗nlp

*This notebook shows how `nlp` can be leveraged to evaluate **Longformer** on **TriviaQA** .*

- The [`nlp`](https://github.com/huggingface/nlp) library allows simple and intuitive access to nlp datasets and metrics.

- **Longformer** is transformer-based model for long-range sequence modeling introduced by *Iz Beltagy, Matthew E. Peters, Arman Cohan* (see paper [here](https://arxiv.org/abs/2004.05150)) and can now be accessed via Transformers via the [docs](https://huggingface.co/transformers/model_doc/longformer.html).

- **TriviaQA** is a reading comprehension dataset containing question-answer-evidence triplets (see paper here [here](https://homes.cs.washington.edu/~eunsol/papers/acl17jcwz.pdf)).

We will evaluate a pretrained `LongformerForQuestionAnswering` model on the *validation* dataset of **TriviaQA**. Along the way, this notebook will show you how `nlp` can be used for effortless preprocessing of data and analysis of the results.

Alright! Let's start by installing the `nlp` library and loading *TriviaQA*. 

### Installs and Imports

In [0]:
# install nlp
!pip install -qq nlp==0.2.0

# Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16:
    import os
    os.kill(os.getpid(), 9)

!pip install -qq transformers==2.11.0

import nlp
import torch

In [0]:
# ATTENTION. Rerunning this command remove the cached trivia qa dataset completely 
!rm -rf /root/.cache/

### Data cleaning and preprocessing 

The total *TriviaQA* dataset has a size of 17 GB once processed.
Downloading and preprocessing the dataset will take around *15 minutes*. ☕
Afterwards the data is serialized in *Arrow* format for quick reloading.



In [0]:
validation_dataset = nlp.load_dataset("trivia_qa", "rc", split="validation[:5%]")

First, let's get an overview of the dataset 🧐

In [18]:
validation_dataset

Dataset(schema: {'question': 'string', 'question_id': 'string', 'question_source': 'string', 'entity_pages': 'struct<doc_source: list<item: string>, filename: list<item: string>, title: list<item: string>, wiki_context: list<item: string>>', 'search_results': 'struct<description: list<item: string>, filename: list<item: string>, rank: list<item: int32>, title: list<item: string>, url: list<item: string>, search_context: list<item: string>>', 'answer': 'struct<aliases: list<item: string>, normalized_aliases: list<item: string>, matched_wiki_entity_name: string, normalized_matched_wiki_entity_name: string, normalized_value: string, type: string, value: string>'}, num_rows: 933)

5% of the validation data corresponds to 933 examples, which we can use as a good snapshot of the actual dataset and get be used to get familiar with the dataset.

Let's check out the datatset's structure.

In [5]:
# check out schema
validation_dataset.schema

question: string not null
question_id: string not null
question_source: string not null
entity_pages: struct<doc_source: list<item: string>, filename: list<item: string>, title: list<item: string>, wiki_context: list<item: string>> not null
  child 0, doc_source: list<item: string>
      child 0, item: string
  child 1, filename: list<item: string>
      child 0, item: string
  child 2, title: list<item: string>
      child 0, item: string
  child 3, wiki_context: list<item: string>
      child 0, item: string
search_results: struct<description: list<item: string>, filename: list<item: string>, rank: list<item: int32>, title: list<item: string>, url: list<item: string>, search_context: list<item: string>> not null
  child 0, description: list<item: string>
      child 0, item: string
  child 1, filename: list<item: string>
      child 0, item: string
  child 2, rank: list<item: int32>
      child 0, item: int32
  child 3, title: list<item: string>
      child 0, item: string
  child 4,

Alright, quite a lot of entries here! For Questions Answering, all we need is the *question*, the *context* and the *answer*. 

The **question** is a single entry, so we keep it.

Because *Longformer* was trained on the Wikipedia part of *TriviaQA*, we will use `validation_dataset["entity_pages"]["search_context"]` as our **context**. 

We can also see that there are multiple entries for the **answer**. In this use case, we define a correct output of the model as one that is one of the answer aliases `validation_dataset["answer"]["aliases"]`. Lastly, we also keep `validation_dataset["answer"]["normalized_value"]`. All other columns can be disregarded. 

We apply the `.map()` function to map the dataset into the format as defined above.

In [19]:
# define the mapping function
def format_dataset(example):
    # the context might be comprised of multiple contexts => me merge them here
    example["context"] = " ".join(("\n".join(example["entity_pages"]["wiki_context"])).split("\n"))
    example["targets"] = example["answer"]["aliases"]
    example["norm_target"] = example["answer"]["normalized_value"]
    return example

# map the dataset and throw out all unnecessary columns
validation_dataset = validation_dataset.map(format_dataset, remove_columns=["search_results", "question_source", "entity_pages", "answer", "question_id"])

933it [00:01, 564.44it/s]


Now, we can check out a first example of the dataset.

In [20]:
validation_dataset[8]

{'context': '',
 'norm_target': 'basket ball',
 'question': 'The Naismith Award is presented in which sport?',
 'targets': ['Basketball',
  'Basketball gear',
  'Bball',
  "Boy's Basketball",
  'B Ball',
  'Shoot hoops',
  'Basketball parity worldwide',
  "Men's Basketball",
  'High school basketball',
  'Basketball Worldwide',
  'Basketball club',
  'B-ball',
  'Basket-ball',
  'Basketball team',
  '🏀',
  'Basketball rim',
  'Basketballer',
  'Rim (basketball)',
  'Basket ball',
  'Basketball net',
  'Baksetball',
  'Basketball player',
  'Basket-Ball',
  "Women's hoops",
  "Men's basketball",
  'BasketBall',
  'Basketball Parity Worldwide',
  'Basket Ball',
  'Baketball',
  'Basketball Player',
  'B ball',
  'Unicycle basketball']}

Great 🙂. That's exactly, the structure we wanted! Some examples might have an empty context so we will filter those examples out.
For this we can use the convenient `.filter()` function of `nlp`.

In [21]:
validation_dataset = validation_dataset.filter(lambda x: len(x["context"]) > 0)
# check out how many samples are left
validation_dataset

100%|██████████| 1/1 [00:00<00:00,  8.28it/s]


Dataset(schema: {'question': 'string', 'context': 'string', 'targets': 'list<item: string>', 'norm_target': 'string'}, num_rows: 467)

Looks like more or less half of our examples have no context and are now filtered out. Let's think about the evaluation on *Longformer* now. 

*Longformer* is able to process inputs of up to a length of **4096** tokens. As a rule of thumb, 4 is the average number of characters per word piece. Therefore, it is a good idea to check for how many examples, the *question* + *context* exceeds 4 * 4096 characters.
Again we can apply the convenient `.map()` function.

In [34]:
print("\n\nLength for each example")
print(30 * "=")

# length for each example
validation_dataset.map(lambda x, i: print(f"Id: {i} - Question Length: {len(x['question'])} - context Length: {len(x['context'])}"), with_indices=True)
print(30 * "=")

print("\n")
print("Num examples larger than 4 * 4096 characters: ")
# filter out examples smaller than 4 * 4096
short_validation_dataset = validation_dataset.filter(lambda x: (len(x['question']) + len(x['context'])) < 4 * 4096)
short_validation_dataset


207it [00:00, 1071.45it/s]



Length for each example
Id: 0 - Question Length: 48 - context Length: 87872
Id: 0 - Question Length: 48 - context Length: 87872
Id: 0 - Question Length: 48 - context Length: 87872
Id: 1 - Question Length: 85 - context Length: 39997
Id: 2 - Question Length: 146 - context Length: 22353
Id: 3 - Question Length: 114 - context Length: 73891
Id: 4 - Question Length: 58 - context Length: 345
Id: 5 - Question Length: 80 - context Length: 36373
Id: 6 - Question Length: 68 - context Length: 1410
Id: 7 - Question Length: 68 - context Length: 115858
Id: 8 - Question Length: 81 - context Length: 1404
Id: 9 - Question Length: 83 - context Length: 65529
Id: 10 - Question Length: 75 - context Length: 32034
Id: 11 - Question Length: 59 - context Length: 24511
Id: 12 - Question Length: 111 - context Length: 46362
Id: 13 - Question Length: 79 - context Length: 26408
Id: 14 - Question Length: 190 - context Length: 52829
Id: 15 - Question Length: 130 - context Length: 28293
Id: 16 - Question Length: 70 -

338it [00:00, 634.34it/s]

Id: 207 - Question Length: 59 - context Length: 39631
Id: 208 - Question Length: 93 - context Length: 23574
Id: 209 - Question Length: 51 - context Length: 22489
Id: 210 - Question Length: 50 - context Length: 78665
Id: 211 - Question Length: 59 - context Length: 25629
Id: 212 - Question Length: 82 - context Length: 174487
Id: 213 - Question Length: 109 - context Length: 16814
Id: 214 - Question Length: 60 - context Length: 237243
Id: 215 - Question Length: 43 - context Length: 166433
Id: 216 - Question Length: 56 - context Length: 105678
Id: 217 - Question Length: 57 - context Length: 86628
Id: 218 - Question Length: 79 - context Length: 115092
Id: 219 - Question Length: 85 - context Length: 226600
Id: 220 - Question Length: 66 - context Length: 123074
Id: 221 - Question Length: 54 - context Length: 144413
Id: 222 - Question Length: 126 - context Length: 83264
Id: 223 - Question Length: 72 - context Length: 79295
Id: 224 - Question Length: 52 - context Length: 12264
Id: 225 - Question

467it [00:00, 760.79it/s]

Id: 415 - Question Length: 55 - context Length: 62348
Id: 416 - Question Length: 54 - context Length: 27761
Id: 417 - Question Length: 29 - context Length: 109259
Id: 418 - Question Length: 28 - context Length: 130367
Id: 419 - Question Length: 27 - context Length: 31985
Id: 420 - Question Length: 90 - context Length: 16250
Id: 421 - Question Length: 48 - context Length: 7822
Id: 422 - Question Length: 57 - context Length: 3912
Id: 423 - Question Length: 40 - context Length: 64161
Id: 424 - Question Length: 75 - context Length: 43874
Id: 425 - Question Length: 116 - context Length: 110238
Id: 426 - Question Length: 45 - context Length: 56151
Id: 427 - Question Length: 88 - context Length: 41601
Id: 428 - Question Length: 113 - context Length: 86389
Id: 429 - Question Length: 39 - context Length: 22368
Id: 430 - Question Length: 80 - context Length: 57896
Id: 431 - Question Length: 114 - context Length: 5022
Id: 432 - Question Length: 70 - context Length: 8883
Id: 433 - Question Length:

Dataset(schema: {'question': 'string', 'context': 'string', 'targets': 'list<item: string>', 'norm_target': 'string'}, num_rows: 114)

Interesting! We can see that only 114 examples have less than 4 * 4096 = 16384 characters...

Most examples seem to have a very long context which will have to be cut to Longformer's maximum length.

### Evaluation

It's time to evaluate *Longformer* on *TriviaQA* 🚀.

Let's write our evaluation function and import the pretrained `LongformerForQuestionAnswering` model. For more details on `LongformerForQuestionAnswering`, see [here](https://huggingface.co/transformers/model_doc/longformer.html?highlight=longformerforquestionanswering#transformers.LongformerForQuestionAnswering).

In [0]:
from transformers import LongformerTokenizerFast, LongformerForQuestionAnswering

tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")

# download the 1.7 GB pretrained model. It might take ~1min
model = LongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")
model.to("cuda")

def evaluate(example):
    def get_answer(question, context):
        # encode question and context so that they are seperated by a tokenizer.sep_token and cut at max_length
        encoding = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=4096)
        input_ids = encoding["input_ids"].to("cuda")
        attention_mask = encoding["attention_mask"].to("cuda")

        # the forward method will automatically set global attention on question tokens
        # The scores for the possible start token and end token of the answer are retrived
        # wrap the function in torch.no_grad() to save memory
        with torch.no_grad():
            start_scores, end_scores = model(input_ids=input_ids, attention_mask=attention_mask)

        # Let's take the most likely token using `argmax` and retrieve the answer
        all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())
        answer_tokens = all_tokens[torch.argmax(start_scores): torch.argmax(end_scores)+1]
        answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))[1:].replace('"', '')  # remove space prepending space token and remove unnecessary '"'
        
        return answer

    # save the model's output here
    example["output"] = get_answer(example["question"], example["context"])

    # save if it's a match or not
    example["match"] = (example["output"] in example["targets"]) or (example["output"] == example["norm_target"])

    return example


We are interested in the performance of the model on short and long samples.
First we evaluate the model on `short_validation_dataset`, which comprises only the samples that are shorter than 4 * 4096 characters.

In [38]:
results_short = results_short.map(evaluate)

114it [00:00, 12381.87it/s]


Now let's check for how many questions we were correct!

In [39]:
print(f"\nNum Correct examples: {sum(results_short['match'])}/{len(results_short)}")
wrong_results = results_short.filter(lambda x: x['match'] is False)
print(f"\nWrong examples: ")
wrong_results.map(lambda x, i: print(f"{i} - Output: {x['output']} - Target: {x['norm_target']}"), with_indices=True)

100%|██████████| 1/1 [00:00<00:00, 135.22it/s]
46it [00:00, 5004.10it/s]


Num Correct examples: 68/114

Wrong examples: 
0 - Output: Doctor Finlay - Target: dr finlay
0 - Output: Doctor Finlay - Target: dr finlay
0 - Output: Doctor Finlay - Target: dr finlay
1 - Output: film industry - Target: film making
2 - Output: kaleidoscopes - Target: kaleidoscope
3 - Output: fruit and vegetables - Target: fruit
4 - Output:  - Target: motel 6
5 - Output:  - Target: fiddler on roof
6 - Output:  - Target: mutiny on bounty
7 - Output: Tom Stoppard - Target: j g ballard
8 - Output: Kentuckian - Target: kentuckian
9 - Output: Roald Amundsen and his party on December 14, 1911. Amundsen named his camp Polheim and the entire plateau surrounding the Pole King Haakon VII Vidde in honour of King Haakon VII of Norway. Robert Falcon Scott returned to Antarctica with his second expedition, the Terra Nova Expedition, initially unaware of Amundsen's secretive expedition. Scott and four other men reached the South Pole on January 17, 1912, thirty-four days after Amundsen. On the retur

Dataset(schema: {'question': 'string', 'context': 'string', 'targets': 'list<item: string>', 'norm_target': 'string', 'output': 'string', 'match': 'bool'}, num_rows: 46)

68/114 - not bad 🔥. Also we can see that many of the wrong outputs are very close to the correct solution or are the correct solution, but just written differently (Number 0,8, ...). One could obviously make a better post processing script that makes sure solutions like 0 and 8 are counted as correct solutions by adding a couple of lines to the `evaluate` function. 

**Note**: *Longformer reached a new SOTA on TriviaQA - see Table 9 in [paper](https://arxiv.org/abs/2004.05150). In order to reproduce the exact results, please refer to the following [instructions](https://github.com/allenai/longformer/blob/master/scripts/cheatsheet.txt).*

Second, we evaluate `LongformerForQuestionAnswering` on the all of the examples.

In [42]:
results = validation_dataset.map(evaluate)

467it [09:31,  1.22s/it]


In [43]:
print(f"Correct examples: {sum(results['match'])}/{len(results)}")

Correct examples: 224/467


Here, we now see a slight degradation. Less than half the samples are correct. It is still a very good score though.

Now you should have all the tools necessary to preprocess your data and evaluate your model with 🤗nlp in no time!

🤗 🤗 **Finish** 🤗🤗

Thanks goes out to Iz Beltagy for proof reading the notebook!